In [1]:

# ============================================================
# 1) Importações e parâmetros
# ============================================================
import os, re, time
import pandas as pd
from glob import glob
from langdetect import detect, DetectorFactory
from transformers import pipeline
import emoji

# Parâmetros para Flask / .env
INPUT_PATH = os.getenv("INPUT_PATH", "data/comentarios_coletados_*.csv")
OUTPUT_DIR = os.getenv("OUTPUT_DIR", "resultados")
LOG_DIR = os.getenv("LOG_DIR", "logs")

# Modelo multilíngue padrão
HF_MODEL = os.getenv("HF_MODEL", "nlptown/bert-base-multilingual-uncased-sentiment")

DetectorFactory.seed = 0
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

print("✅ Parâmetros carregados")
print("INPUT_PATH:", INPUT_PATH)
print("OUTPUT_DIR:", OUTPUT_DIR)
print("LOG_DIR:", LOG_DIR)
print("HF_MODEL:", HF_MODEL)

c:\Users\renan\OneDrive\Desktop\youtube-sentiment-analysis\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Parâmetros carregados
INPUT_PATH: data/comentarios_coletados_*.csv
OUTPUT_DIR: resultados
LOG_DIR: logs
HF_MODEL: nlptown/bert-base-multilingual-uncased-sentiment


In [2]:

# ============================================================
# 2) Carregar último CSV de coleta
# ============================================================
def ultimo_csv(pattern: str) -> str:
    arquivos = sorted(glob(pattern))
    if not arquivos:
        raise FileNotFoundError("Nenhum arquivo de coleta encontrado. Execute a Fase 1 antes.")
    return arquivos[-1]

if "*" in INPUT_PATH:
    INPUT_PATH = ultimo_csv(INPUT_PATH)

df = pd.read_csv(INPUT_PATH)
print(f"📥 Carregado: {INPUT_PATH} | Linhas: {len(df)}")
df.head(5)

📥 Carregado: data\comentarios_coletados_20251030_193635.csv | Linhas: 4370


,video_id,video_titulo,comentario,idioma,data_coleta
0,VKf6NF0OD5A,The new and improved YouTube Studio is here,Check out the entire series on how to use YouT...,en,2025-10-30
1,VKf6NF0OD5A,The new and improved YouTube Studio is here,Wow❤,pl,2025-10-30
2,VKf6NF0OD5A,The new and improved YouTube Studio is here,Nice,ro,2025-10-30
3,VKf6NF0OD5A,The new and improved YouTube Studio is here,मेरा चैनल पर 100k सब्सक्राइबर है अभी तक सिल्व...,hi,2025-10-30
4,VKf6NF0OD5A,The new and improved YouTube Studio is here,❤❤❤❤❤,indefinido,2025-10-30


In [3]:

# ============================================================
# 3) Limpeza e normalização dos textos
# ============================================================
def limpar_texto(texto: str) -> str:
    t = emoji.demojize(str(texto))
    t = re.sub(r"http\S+|www\.\S+", " ", t)
    t = re.sub(r"@[A-Za-z0-9_]+|#[A-Za-z0-9_]+", " ", t)
    t = re.sub(r"[^A-Za-zÀ-ÿ\s:]", " ", t)
    t = re.sub(r"\s+", " ", t).strip().lower()
    return t

df["comentario_limpo"] = df["comentario"].astype(str).apply(limpar_texto)
df = df[df["comentario_limpo"].str.len() > 3].copy()
print("🧹 Limpeza concluída | Linhas após limpeza:", len(df))
df[["video_titulo","comentario","comentario_limpo"]].head(5)

🧹 Limpeza concluída | Linhas após limpeza: 4007


,video_titulo,comentario,comentario_limpo
0,The new and improved YouTube Studio is here,Check out the entire series on how to use YouT...,check out the entire series on how to use yout...
1,The new and improved YouTube Studio is here,Wow❤,wow:red heart:
2,The new and improved YouTube Studio is here,Nice,nice
4,The new and improved YouTube Studio is here,❤❤❤❤❤,:red heart::red heart::red heart::red heart::r...
5,The new and improved YouTube Studio is here,Good❤❤❤❤ 0:25,good:red heart::red heart::red heart::red hear...


In [4]:

# ============================================================
# 4) Detecção de idioma (sem filtragem)
# ============================================================
def detectar_idioma_seguro(texto: str) -> str:
    try:
        return detect(texto)
    except Exception:
        return "indefinido"

df["idioma"] = df["comentario_limpo"].apply(detectar_idioma_seguro)
idiomas_contagem = df["idioma"].value_counts().head(5)
print("🌍 Top 5 idiomas detectados:")
print(idiomas_contagem)
df.head(5)

🌍 Top 5 idiomas detectados:
idioma
en    2914
id     121
es      98
so      86
pt      72
Name: count, dtype: int64


,video_id,video_titulo,comentario,idioma,data_coleta,comentario_limpo
0,VKf6NF0OD5A,The new and improved YouTube Studio is here,Check out the entire series on how to use YouT...,en,2025-10-30,check out the entire series on how to use yout...
1,VKf6NF0OD5A,The new and improved YouTube Studio is here,Wow❤,en,2025-10-30,wow:red heart:
2,VKf6NF0OD5A,The new and improved YouTube Studio is here,Nice,pl,2025-10-30,nice
4,VKf6NF0OD5A,The new and improved YouTube Studio is here,❤❤❤❤❤,en,2025-10-30,:red heart::red heart::red heart::red heart::r...
5,VKf6NF0OD5A,The new and improved YouTube Studio is here,Good❤❤❤❤ 0:25,en,2025-10-30,good:red heart::red heart::red heart::red hear...


In [5]:
# ============================================================
# 5) Pipeline HuggingFace (multilíngue) — robusto e em lotes
# ============================================================
import math
from transformers import pipeline

# --- VERIFICAÇÃO DE DEPENDÊNCIAS ---------------------------------------------
missing = []
try:
    import torch  # noqa: F401
except Exception:
    missing.append("torch")
try:
    import transformers  # noqa: F401
except Exception:
    missing.append("transformers")

if missing:
    raise RuntimeError(
        "Dependências ausentes: "
        + ", ".join(missing)
        + ". Instale com: pip install torch transformers"
    )

# --- PREPARO DOS DADOS --------------------------------------------------------
# Garante que temos textos para analisar
serie_textos = df["comentario_limpo"].dropna().astype(str)
serie_textos = serie_textos[serie_textos.str.len() > 3]  # filtra muito curtos

if serie_textos.empty:
    raise ValueError("Nenhum comentário válido para análise após limpeza.")

textos = serie_textos.tolist()

# --- CARREGAMENTO DO MODELO ---------------------------------------------------
HF_MODEL = os.getenv("HF_MODEL", "nlptown/bert-base-multilingual-uncased-sentiment")
print(f"🔁 Carregando modelo: {HF_MODEL}")

try:
    sentiment_pipe = pipeline(
        task="sentiment-analysis",
        model=HF_MODEL,
        truncation=True,
    )
except Exception as e:
    raise RuntimeError(
        "Falha ao carregar o modelo do HuggingFace.\n"
        "Causas comuns: sem internet, proxy/firewall, cache corrompido.\n"
        f"Modelo: {HF_MODEL}\nErro: {repr(e)}\n\n"
        "Soluções rápidas:\n"
        " - Verifique sua conexão e proxy\n"
        " - Se necessário, faça login: huggingface-cli login\n"
        " - Atualize/instale dependências: pip install -U transformers torch\n"
    )

# --- INFERÊNCIA EM LOTES ------------------------------------------------------
def estrela_para_polaridade(label: str) -> str:
    # label típico: '1 star', '2 stars', ..., '5 stars'
    try:
        stars = int(label.split()[0])
    except Exception:
        return "neutro"
    if stars <= 2:
        return "negativo"
    elif stars == 3:
        return "neutro"
    else:
        return "positivo"

def inferir_em_batches(pipe, textos, batch_size=64):
    resultados = []
    total = len(textos)
    num_batches = math.ceil(total / batch_size)
    for b in range(num_batches):
        ini = b * batch_size
        fim = min((b + 1) * batch_size, total)
        lote = textos[ini:fim]
        try:
            preds = pipe(lote)
            resultados.extend(preds)
        except Exception as e:
            # Se um lote falhar, tenta item a item para isolar problemáticos
            print(f"⚠️ Falha no lote {b+1}/{num_batches}: {repr(e)}. Tentando item a item…")
            for t in lote:
                try:
                    resultados.append(pipe(t)[0])
                except Exception as ee:
                    # Se ainda falhar, marca como neutro
                    print(f"  ↳ Comentário com erro ignorado: {repr(ee)}")
                    resultados.append({"label": "3 stars", "score": 0.0})
    return resultados

preds = inferir_em_batches(sentiment_pipe, textos, batch_size=64)

# Recria um DataFrame alinhado ao índice original pós-limpeza
df_validos = serie_textos.to_frame().copy()
df_validos["pred_label"] = [p.get("label", "3 stars") for p in preds]
df_validos["score"] = [float(p.get("score", 0.0)) for p in preds]
df_validos["sentimento"] = df_validos["pred_label"].apply(estrela_para_polaridade)
df_validos["modelo_utilizado"] = HF_MODEL

# Junta de volta no df principal pelos índices
df = df.join(df_validos[["sentimento", "score", "modelo_utilizado"]], how="left")

print("✅ Análise concluída.")
display(df[["video_titulo","comentario_limpo","idioma","sentimento","score"]].head(10))


🔁 Carregando modelo: nlptown/bert-base-multilingual-uncased-sentiment


Device set to use cpu


✅ Análise concluída.


,video_titulo,comentario_limpo,idioma,sentimento,score
0,The new and improved YouTube Studio is here,check out the entire series on how to use yout...,en,positivo,0.484485
1,The new and improved YouTube Studio is here,wow:red heart:,en,positivo,0.686705
2,The new and improved YouTube Studio is here,nice,pl,positivo,0.436841
4,The new and improved YouTube Studio is here,:red heart::red heart::red heart::red heart::r...,en,positivo,0.315854
5,The new and improved YouTube Studio is here,good:red heart::red heart::red heart::red hear...,en,positivo,0.399166
6,The new and improved YouTube Studio is here,nice:red heart::red heart::red heart::red hear...,en,positivo,0.485567
7,The new and improved YouTube Studio is here,great tips :hundred points:,en,positivo,0.787476
8,The new and improved YouTube Studio is here,vreri nice :face with open mouth::face with op...,en,positivo,0.385908
9,The new and improved YouTube Studio is here,great information :ok hand light skin tone::ok...,en,positivo,0.417952
10,The new and improved YouTube Studio is here,thanks for subscribers :smiling face with smil...,en,positivo,0.255855


In [6]:

# ============================================================
# 6) Exportação com timestamp + relatório multilíngue + histórico
# ============================================================
from IPython.display import Markdown, display

ts = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
out_csv = os.path.join(OUTPUT_DIR, f"comentarios_analisados_{ts}.csv")
df.to_csv(out_csv, index=False, encoding="utf-8-sig")

# Distribuição de sentimentos
total = len(df)
dist = df["sentimento"].value_counts(normalize=True).reindex(["positivo","neutro","negativo"]).fillna(0.0)
pct_pos, pct_neu, pct_neg = dist["positivo"]*100, dist["neutro"]*100, dist["negativo"]*100

# Top 5 idiomas
idiomas_top = df["idioma"].value_counts(normalize=True).head(5)*100

idiomas_str = "\n".join([f"  - {idioma}: **{pct:.1f}%**" for idioma, pct in idiomas_top.items()])

# Relatório em Markdown
report_md = f"""
## 📊 Relatório Automático — {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}
- **Modelo utilizado:** `{HF_MODEL}`  
- **Arquivo de entrada:** `{INPUT_PATH}`  
- **Total de comentários analisados:** **{total}**
- **Distribuição de sentimentos:**  
  - Positivo: **{pct_pos:.1f}%**  
  - Neutro: **{pct_neu:.1f}%**  
  - Negativo: **{pct_neg:.1f}%**
- **Top 5 idiomas detectados:**  
{idiomas_str}
- **Arquivo salvo:** `{out_csv}`
"""
display(Markdown(report_md))

# Histórico
hist_path = os.path.join(LOG_DIR, "analises.csv")
entry = pd.DataFrame([{
    "data_hora": pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
    "modelo_utilizado": HF_MODEL,
    "arquivo_entrada": INPUT_PATH,
    "total_comentarios": total,
    "pct_positivo": round(float(pct_pos), 3),
    "pct_neutro": round(float(pct_neu), 3),
    "pct_negativo": round(float(pct_neg), 3),
    "idiomas_top5": ", ".join([f"{idioma}:{pct:.1f}%" for idioma, pct in idiomas_top.items()]),
    "arquivo_saida": out_csv,
}])

if os.path.exists(hist_path):
    entry.to_csv(hist_path, mode="a", index=False, header=False, encoding="utf-8-sig")
else:
    entry.to_csv(hist_path, index=False, encoding="utf-8-sig")

print("📦 Resultado salvo:", out_csv)
print("🗂️ Histórico atualizado:", hist_path)


## 📊 Relatório Automático — 2025-10-30 20:09:39
- **Modelo utilizado:** `nlptown/bert-base-multilingual-uncased-sentiment`  
- **Arquivo de entrada:** `data\comentarios_coletados_20251030_193635.csv`  
- **Total de comentários analisados:** **4007**
- **Distribuição de sentimentos:**  
  - Positivo: **62.6%**  
  - Neutro: **13.7%**  
  - Negativo: **23.8%**
- **Top 5 idiomas detectados:**  
  - en: **72.7%**
  - id: **3.0%**
  - es: **2.4%**
  - so: **2.1%**
  - pt: **1.8%**
- **Arquivo salvo:** `resultados\comentarios_analisados_20251030_200939.csv`


📦 Resultado salvo: resultados\comentarios_analisados_20251030_200939.csv
🗂️ Histórico atualizado: logs\analises.csv
